In [1]:
import numpy as np
import pandas as pd
from time import time
from sklearn import metrics
from collections import defaultdict
import matplotlib.pyplot as plt
import glob


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.metrics.cluster import contingency_matrix, homogeneity_score, completeness_score, adjusted_rand_score, adjusted_mutual_info_score, v_measure_score
from sklearn.metrics.cluster import contingency_matrix

## Import libraries
import re
import nltk
from nltk import pos_tag
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords' )

import string
from string import punctuation
import matplotlib.pyplot as plt

In [2]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
import random
import pandas as pd
import numpy as np
import re, json, datetime, pytz
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from collections import Counter
import nltk

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
import pytz
import datetime

pst_tz = pytz.timezone('America/Los_Angeles')

In [4]:
from happytransformer import HappyTextClassification
happy_tc = HappyTextClassification(model_type="DISTILBERT", 
                                       model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)

03/13/2022 03:55:41 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [5]:
from transformers import pipeline
pipe = pipeline(task='text2text-generation', model='facebook/m2m100_418M')

In [6]:
def transformer_sentiment(tweet):
    
    result = happy_tc.classify_text(tweet)
    if result.label == 'NEGATIVE':
        return -1, -result.score
    else:
        return 1, result.score
    

In [7]:
def translate_and_transformer_sentiment(tweet):
    
    tweet = pipe(tweet, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='en'))
    tweet = tweet[0]['generated_text']
    result = happy_tc.classify_text(tweet)
    if result.label == 'NEGATIVE':
        return -1, -result.score
    else:
        return 1, result.score

In [8]:
def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [54]:
from_time = datetime.datetime.strptime('2015-02-01 15:00:00', '%Y-%m-%d %H:%M:%S')
to_time =  datetime.datetime.strptime('2015-02-02 10:00:00', '%Y-%m-%d %H:%M:%S')

def get_data(text, min_retweet=10):
    with open(text+'.txt', 'r',encoding="utf8") as f:
        retweet_count, fav_count, followers, place, title, time = [],[],[],[],[],[]
        sentiment_polarity, sentiment_score = [], []
        pst_tz = pytz.timezone('US/Pacific') 
        for line in f:
            tweet = json.loads(line)
            time_now = datetime.datetime.fromtimestamp(tweet['citation_date'])
            retweet_now = tweet['metrics']['citations']['total']
            if time_now > from_time and time_now < to_time and retweet_now >= min_retweet:
                title.append(tweet['title'])
                retweet_count.append(tweet['metrics']['citations']['total']) 
                fav_count.append(tweet['tweet']['favorite_count'])
                followers.append(tweet['author']['followers'])
                time.append(datetime.datetime.fromtimestamp(tweet['citation_date'], pst_tz))
                place.append(tweet['tweet']['user']['location'])
                cleaned_tweet = clean_tweet(tweet['title'])
                if not tweet['tweet']['lang'] == 'en':
                    senti_polarity, senti_score = translate_and_transformer_sentiment(cleaned_tweet)
                    sentiment_polarity.append(senti_polarity)
                    sentiment_score.append(senti_score)
                else:
                    senti_polarity, senti_score = transformer_sentiment(cleaned_tweet)
                    sentiment_polarity.append(senti_polarity)
                    sentiment_score.append(senti_score)
    d = {'title':title, 'retweet_count':retweet_count, 'favourite_count': fav_count, 'followers':followers,
         'time':time,'place':place, 'sentiment_polarity':sentiment_polarity, 'sentiment_score':sentiment_score}
    df = pd.DataFrame(d)
    return df   

In [47]:
file1 = open('tweets_#gopatriots.txt', 'r')
lines = file1.readlines()

In [49]:
count = 0
for line in lines:
    json_object = json.loads(line)
    if json_object['metrics']['citations']['total'] >= 10:
        count = count + 1
print(count)

122


In [55]:
df_patriots = get_data('tweets_#gopatriots', 10)

/home/shivam/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:997: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [56]:
df_patriots.head(40)

,title,retweet_count,favourite_count,followers,time,place,sentiment_polarity,sentiment_score
0,Tom Brady is an American Legend. #BeatTheSeaha...,70,0,190.0,2015-02-01 16:08:03-08:00,Boston,1,0.996323
1,Life of a @Patriots fan these days. #patsnatio...,103,0,98.0,2015-02-02 06:28:38-08:00,,1,0.903411
2,Today is THE Day. #GoPatriots http://t.co/nFKn...,69,0,1372.0,2015-02-01 16:41:17-08:00,Boston,1,0.998299
3,"In honour of the Super Bowl, he's a really sup...",11,3,107.0,2015-02-01 22:52:12-08:00,London. The World.,1,0.999739
4,Me and @JJUJWorld couldn't make it to the @Sup...,371,0,330.0,2015-02-01 21:17:43-08:00,Mother/Model /London/USA,-1,-0.985524
5,"#GoPatriots Brady,Gray, Ridley, Bolder y Revis...",19,0,95.0,2015-02-01 21:47:40-08:00,Mexico City,-1,-0.939634
6,@NewEnglandPats4 at my happy place #GoPatriots...,35,0,621.0,2015-02-01 16:33:51-08:00,"Toronto, Canada",1,0.999806
7,Torcida uniformizada! #GoPatriots! #ESPNtemSup...,24,0,2.0,2015-02-01 16:05:56-08:00,São Paulo,-1,-0.974622
8,"Hoy, por la historia!!!!!! Nadie dijo que serí...",165,0,50.0,2015-02-01 20:26:39-08:00,Mexico,-1,-0.971080
9,Uno haciéndole barra a @_El_Patriota y el tipo...,14,0,88.0,2015-02-01 16:10:59-08:00,"Bogotá, D.C. - Colombia",-1,-0.991579


In [ ]:
# df_patriots.to_pickle("df_patriots.pkl")
# df_patriots = pd.read_pickle("df_patriots.pkl")

In [ ]:
df_hawks = get_data('tweets_#gohawks', 15)

In [ ]:
# df_hawks.to_pickle("df_hawks.pkl")
# df_hawks = pd.read_pickle("df_hawks.pkl")